<a href="https://colab.research.google.com/github/Sarthakpunj/Food_label_reader_2/blob/main/Food_label_redaer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:


# Step 2: Import libraries
from PIL import Image
import pytesseract
from google.colab import files

# Step 3: Upload an image
uploaded = files.upload()

# Step 4: Load the image
image_path = next(iter(uploaded))  # Get the uploaded file name
image = Image.open(image_path)

# Step 5: Extract text from image
extracted_text = pytesseract.image_to_string(image)

# Step 6: Print the extracted text
print("Extracted Text:")
print(extracted_text)


Saving calorie-count-label.jpg to calorie-count-label.jpg
Extracted Text:
   

(RGREDIENTE: WATER VEGETABLE OR MLBID (00 SOM 0. PRY |
OROGERATED SOE Cy. RATURA. SET CC REALE
GELATIR, VEGETABLE MONO MID) OGLYOERECS, MO. Sit
(POTASSRIM SORBATE, CALCU DISORRG EU LED HOE EAL
JCTE AOD, ATU. ID TCR RA PRE

ORTTBE FRCL, e

ae



In [23]:
from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

In [24]:
from google.colab import userdata
api= userdata.get('gemini')

llm = ChatGoogleGenerativeAI(google_api_key=api,model="gemini-2.0-flash")

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """"
            You are a food safety analysis assistant. A user has submitted text from the back of a food product. It may include nutrition facts, marketing text, and an ingredient list.

Your tasks are:

Extract only the ingredient list from the text. Look for a section starting with “Ingredients:” and focus only on the ingredients mentioned there.

Analyze the ingredient list for potentially harmful or controversial substances, including:

Artificial preservatives (e.g., BHT, BHA, Sodium Benzoate)

Artificial colors (e.g., Red 40, Yellow 5)

High levels of added sugars or sweeteners (e.g., HFCS, Aspartame)

Processed oils (e.g., partially hydrogenated oils, palm oil)

Known allergens or irritants (e.g., soy, gluten, dairy) — only flag if relevant

Determine whether the product is harmful for human consumption based on current scientific and regulatory knowledge (FDA, WHO, EWG). If any ingredients raise concern, explain clearly why they are considered harmful.

If no concerns are found, state that the product appears safe for consumption and explain why.

Keep your answer simple and easy to understand. Use clear, factual reasoning. Do not guess or make assumptions if data is unavailable.

            """,
        ),
        ("placeholder","{chat_history}"),
        ("human","{query}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [27]:
from langchain.agents import create_tool_calling_agent,AgentExecutor

In [28]:
tools = []
agent = create_tool_calling_agent(
    llm = llm,
    prompt = prompt,
    tools = tools
)


In [29]:
agent_exc= AgentExecutor(agent=agent,tools=tools,verbose=True)
query = extracted_text
raw_res = agent_exc.invoke({"query": query})
print(raw_res)



> Entering new AgentExecutor chain...
Ingredients: WATER, VEGETABLE OIL BLEND (SOYBEAN OIL, FULLY HYDROGENATED SOYBEAN OIL), SALT, CONTAINS LESS THAN 2% OF: GELATIN, VEGETABLE MONO- AND DIGLYCERIDES, MONOSODIUM GLUTAMATE, XANTHAN GUM, PHOSPHORIC ACID, POTASSIUM SORBATE, CALCIUM DISODIUM EDTA (TO PROTECT FLAVOR), VITAMIN A PALMITATE, BETA CAROTENE (COLOR), NATURAL FLAVOR.

Analysis:

The ingredient list contains the following items of concern:

Monosodium Glutamate (MSG): Some individuals are sensitive to MSG, experiencing symptoms such as headaches, flushing, sweating, and other reactions.

Potassium Sorbate and Calcium Disodium EDTA: These are preservatives. While generally recognized as safe in small amounts, some people prefer to avoid them.

Fully Hydrogenated Soybean Oil: Contains trans fats, which are associated with increased risk of heart disease.

Recommendation:

The product contains ingredients that may be harmful to certain individuals or have negative health impacts when